<a href="https://colab.research.google.com/github/SilvanaJ90/udemy_-machine_learning_ds/blob/main/Generative_adversarial_Networks_modificacion_videos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Modificación de vídeos (cambio de cara) utilizando Generative Adversarial Networks

<div style="background-color:#D9EEFF;color:black;padding:2%;">
<h2>Enunciado del caso práctico</h2>

En este caso práctico, se propone al alumno el uso de Generative Adversarial Networks (GANs) para modificar un vídeo e intercambiar la cara de una persona.

Para este caso concreto se prompone el uso de un técnica/arquitectura de Generative Adversarial Network conocida como [StyleGan2](https://github.com/NVlabs/stylegan2) y más concretamente un desarrollo influenciado por esta Red Neuronal Artificial Profunda que se denomina [Stitch in Time](https://stitch-time.github.io/).

</div>

## Contexto adicional sobre Generative Adversarial Networks

Desde el año en el que surgieron este tipo de Redes Neuronales Artificiales Profundas no han parado de publicarse [aplicaciones muy interesantes y modificaciones de su arquitectura inicial](https://arxiv.org/abs/2008.02793).

En términos generales, podríamos clasificar las GANs en varias categorías:

**1. GANs no condicionales y condicionales**

* **No condicionales**: El generador convierte una entrada de ruido en una imagen falsa, y el discriminador diferencia entre imágenes reales e imágenes falsas. No hay señales de control adicionales para guiar el proceso de generación. Un ejemplo muy interesante es: https://thispersondoesnotexist.com/

* **Condicionales**: El generador recibe una señal de control adicional como entrada, que podría ser otra imagen, texto o una etiqueta categórica. El objetivo del generador es producir salidas que correspondan a la señal de control proporcionada. Por ejemplo, si la señal de control es una etiqueta de categoría, el generador debería producir una imagen que pertenezca a esa categoría específica. El discriminador también tiene en cuenta la señal de control para diferenciar entre imágenes reales e imágenes generadas.

**2. Tipos de GANs Basados en su Arquitectura**

* **GAN Original**: Introducido por Goodfellow et al., establece la estructura básica de las GANs.
* **DCGAN (Deep Convolutional GAN)**: Introduce capas convolucionales en las GANs para mejorar la calidad de las imágenes generadas.
* **CoGAN (Coupled GANs)**: Utiliza múltiples GANs entrenadas juntas para mejorar la generación de imágenes.
* **PgGAN (Progressive Growing GAN)**: Aumenta progresivamente la resolución de las imágenes generadas durante el entrenamiento para mejorar la calidad.
* **StyleGAN**: Introduce un enfoque basado en estilos para la generación de imágenes, permitiendo un control más fino sobre las características generadas.

**3. Aplicaciones de las GANs**
* **Síntesis Semántica de Imágenes**: Conversión de representaciones semánticas editables por humanos a imágenes fotorrealistas.
* **Traducción de Imágenes**: Traducción de imágenes de un dominio a otro.
* **Restauración de Imágenes, Superresolución e Inpainting**: Transformación de distribuciones de imágenes para mejorar la calidad visual.
* **Síntesis de Video**: Generación y manipulación de videos.
* **Renderizado Neural**: Uso de redes neuronales para mejorar los procesos de renderizado gráfico.



# Resolución del caso práctico

## 0. Instalación de librerías externas

Lo primero que vamos a necesitar para resolver este caso práctico es descargar la Generative Adversarial Network del repositorio de Github: https://github.com/rotemtzaban/STIT y subir este repositorio a Google Drive después de aplicarle algunas configuraciones.

In [18]:
!pip install -r /content/drive/MyDrive/dataset/STIT/requirements.txt
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-rdr92y3h
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-rdr92y3h
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


## 2. División del vídeo en frames

A continuación, debemos dividr el vídeo en un conjunto de imágenes fíjas que pueda editar la Red Neuronal Artificial y después volver a componer en forma de un vídeo nuevo.

El vídeo que yo he creado tiene una duración de 7 segundos y 30 fps.

In [21]:
!ffmpeg -i "/content/drive/MyDrive/dataset/STIT/IMG_0138.mov" -vf "scale=720:-1" "/content/drive/MyDrive/dataset/STIT/silvana_frames/out%04d.png"

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

## 2. Fine-tuning del modelo

El entrenamiento de las Generative Adversarial Networks es un proceso extremadamente costoso y que requiere grandes recursos computacionales.

Por este motivo, vamos a aplicar la técnica de re-entrenamiento (fine-tuning) que hemos presentado en secciones anteriores.

En este caso el fine-tuning lo aplicamos sobre un modelo base ya entrenado con caras de personas.

In [22]:
!python /content/drive/MyDrive/dataset/STIT/train.py --input_folder /content/drive/MyDrive/dataset/STIT/silvana_frames \
 --output_folder /content/drive/MyDrive/dataset/STIT/silvana/train_results \
 --run_name silvana \
 --num_pti_steps 3

Number of images: 91
Aligning images
100% 91/91 [00:15<00:00,  5.86it/s]
100% 91/91 [00:05<00:00, 17.60it/s]
Aligning completed
Loading e4e over the pSp framework from checkpoint: /content/drive/MyDrive/dataset/STIT/pretrained_models/e4e_ffhq_encode.pt
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alex

## 3. Generación/Modificación del vídeo

LLegados a este punto ya tenemos todo listo para utilizar nuestra Red Neuronal Artificial con Fine-tuning para realizar la modificación del vídeo.

In [23]:
# Generación de un vídeo donde modifica la edad
!python /content/drive/MyDrive/dataset/STIT/edit_video.py --input_folder /content/drive/MyDrive/dataset/STIT/silvana_frames \
 --output_folder /content/drive/MyDrive/dataset/STIT/edits/silvana_joven \
 --run_name silvana \
 --edit_name age \
 --edit_range 8 8 1

100% 91/91 [00:05<00:00, 17.63it/s]
  0% 0/91 [00:00<?, ?it/s]Setting up PyTorch plugin "bias_act_plugin"... /usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:1967: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... /usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:1967: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Done.
100% 91/91 [00:27<00:00,  3.26it/s]


In [ ]:
# Generación de un vídeo donde elimina la sonrisa
!python /content/drive/MyDrive/dataset/STIT/edit_video_stitching_tuning.py --input_folder /content/drive/MyDrive/dataset/STIT/video_frames \
 --output_folder /content/drive/MyDrive/dataset/STIT/edits/silvana_viejo \
 --run_name silvana \
 --edit_name smile \
 --edit_range -3 -3 1 \
 --outer_mask_dilation 50